In [ ]:
import os
import pytesseract
from pdf2image import convert_from_path
import requests
import json
from sentence_transformers import SentenceTransformer
import re
import weaviate
from weaviate import client
import weaviate.classes as wvc
from weaviate.classes.config import Property, DataType
import ollama
from weaviate.classes.init import AdditionalConfig, Timeout
from weaviate.util import generate_uuid5
import json

load all JSON files at once

In [22]:
def CreateCollectionAndLoad():

    client = weaviate.connect_to_local(
        port=8080,
        grpc_port=50051,
        additional_config=AdditionalConfig(
            timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
        )
    )
    print(f"Client: {weaviate.__version__}, Server: client.get_meta().get('version')")
    collection_name = "Notices"
    client.collections.delete(collection_name)
    collection = client.collections.create(
            name=collection_name,
            vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_ollama(
                api_endpoint="http://host.docker.internal:11434",
                model="snowflake-arctic-embed"
            ),
            generative_config=wvc.config.Configure.Generative.ollama(
                api_endpoint="http://host.docker.internal:11434",  
                model="llama3.2"
            )
        )


    folder_path = "downloaded_data"

    # List all files in the folder
    files_in_folder = os.listdir(folder_path)

    # Filter files based on the extension (e.g., JSON files)
    json_files = [f for f in files_in_folder if f.endswith(".JSON")]
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)

        with open(file_path, "r") as json_file:
            data = json.load(json_file)
            data = data["opportunitiesData"]

        with collection.batch.dynamic() as batch:
            for d in data:
                batch.add_object({
                    "noticeId": d["noticeId"],
                    "title": d["title"],
                    "fullParentPathName": d["fullParentPathName"],
                    "fullParentPathCode": d["fullParentPathCode"],
                    "postedDate": d["postedDate"],
                    "type": d["type"],
                    "baseType": d["baseType"],
                    "archiveType": d["archiveType"],
                    "archiveDate": d["archiveDate"],
                    "typeOfSetAsideDescription": d["typeOfSetAsideDescription"],
                    "typeOfSetAside": d["typeOfSetAside"],
                    "responseDeadLine": d["responseDeadLine"],
                    "naicsCode": d["naicsCode"],
                    "classificationCode": d["classificationCode"],
                    "active": d["active"],
                    "award": d["award"],
                    "awapointOfContactrd": d["pointOfContact"],
                    "description": d["description"],
                    "organizationType": d["organizationType"],
                    "officeAddress": d["officeAddress"],
                    #"placeOfPerformance": d["placeOfPerformance"],
                    "additionalInfoLink": d["additionalInfoLink"],
                    "uiLink": d["uiLink"],
                    "links": d["links"],
                    #"resourceLinks": d["resourceLinks"],

                })
    for i in collection.iterator():
        print(i.properties)

CreateCollectionAndLoad()

Client: 4.10.4, Server: client.get_meta().get('version')
{'postedDate': '2025-01-23', 'description': 'https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=a5dee0941d8545cb884c2d262b0e7df9', 'type': 'Justification', 'award': {'awardee': {'name': '', 'ueiSAM': '', 'duns': '', 'location': {'zip': '', 'streetAddress2': '', 'state': {'name': '', 'code': ''}, 'streetAddress': '', 'city': {'name': '', 'code': ''}, 'country': {'name': '', 'code': ''}}, 'manual': False}, 'number': 'N6600121D0161', 'date': '2025-01-22', 'amount': ''}, 'naicsCode': '541330', 'fullParentPathCode': '017.1700.SPAWAR.SPAWAR PACIFIC.N66001', 'officeAddress': {'state': 'CA', 'zipcode': '92152-5001', 'city': 'SAN DIEGO', 'countryCode': 'USA'}, 'uiLink': 'https://sam.gov/opp/a5dee0941d8545cb884c2d262b0e7df9/view', 'archiveType': 'autocustom', 'classificationCode': 'A', 'additionalInfoLink': None, 'active': 'Yes', 'organizationType': 'OFFICE', 'links': [{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunit

In [ ]:
client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=AdditionalConfig(
        timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
    ))
collection = client.collections.get("Notices")

response = collection.generate.near_text(
    query = "Architect-Engineer (A-E) Type C Services ",
    limit=2,
    single_prompt="just print {title} and {noticeId}",
    
)
for i in response.objects:
    print(i.generated)

In [28]:
client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=AdditionalConfig(
        timeout=Timeout(init=60, query=120, insert=240)  # Values in seconds
    ))
collection = client.collections.get("Notices")
print(len(list(collection.iterator())))
print(list(collection.iterator()))
client.close()


108
[Object(uuid=_WeaviateUUIDInt('009fbc2c-e540-4a0d-ad94-5c0b791b5f75'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'postedDate': '2025-01-23', 'description': 'https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=a5dee0941d8545cb884c2d262b0e7df9', 'type': 'Justification', 'naicsCode': '541330', 'award': {'awardee': {'manual': False, 'name': '', 'duns': '', 'location': {'zip': '', 'streetAddress2': '', 'state': {'name': '', 'code': ''}, 'streetAddress': '', 'city': {'code': '', 'name': ''}, 'country': {'name': '', 'code': ''}}, 'ueiSAM': ''}, 'number': 'N6600121D0161', 'date': '2025-01-22', 'amount': ''}, 'fullParentPathCode': '017.1700.SPAWAR.SPAWAR PACIFIC.N66001', 'additionalInfoLink': None, 'uiLink': 'https://sam.gov/opp/a5dee0941d8545cb884c2d262b0e7df9/view', 'archiveType': 'autocustom', 'placeOfPerformance': None, 'officeAddress': {'

monitoring


In [29]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class FileMonitorHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return  # Ignore directories
        print(f"New file detected: {event.src_path}")
        self.run_code(event.src_path)

    def run_code(self, file_path):
        # Your code to handle the new file goes here
        print(f"Running code for the file: {file_path}")
        # For example, process the file
        client = weaviate.connect_to_local(
            port=8080,
            grpc_port=50051,
            additional_config=AdditionalConfig(
                timeout=Timeout(init=30, query=60, insert=120)  # Values in seconds
            )
        )
        print(f"Client: {weaviate.__version__}, Server: client.get_meta().get('version')")
        collection_name = "Notices"
        #client.collections.delete(collection_name)
        #collection = client.collections.create(
         #       name=collection_name,
          #      vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_ollama(
           #         api_endpoint="http://host.docker.internal:11434",
            #        model="snowflake-arctic-embed"
             #   ),
              #  generative_config=wvc.config.Configure.Generative.ollama(
               #     api_endpoint="http://host.docker.internal:11434",  
                #    model="llama3.2"
                #))
        file_path = os.path.normpath(file_path)


        #file_path = "downloaded_data/01-23-2025_541330.JSON"

        with open(file_path, "r") as json_file:
            data = json.load(json_file)
            data = data["opportunitiesData"]
        print(data)
        collection = client.collections.get(collection_name)
        with collection.batch.dynamic() as batch:
            for d in data:
                batch.add_object({
                    "noticeId": d["noticeId"],
                    "title": d["title"],
                    "fullParentPathName": d["fullParentPathName"],
                    "fullParentPathCode": d["fullParentPathCode"],
                    "postedDate": d["postedDate"],
                    "type": d["type"],
                    "baseType": d["baseType"],
                    "archiveType": d["archiveType"],
                    "archiveDate": d["archiveDate"],
                    "typeOfSetAsideDescription": d["typeOfSetAsideDescription"],
                    "typeOfSetAside": d["typeOfSetAside"],
                    "responseDeadLine": d["responseDeadLine"],
                    "naicsCode": d["naicsCode"],
                    "classificationCode": d["classificationCode"],
                    "active": d["active"],
                    "award": d["award"],
                    "awapointOfContactrd": d["pointOfContact"],
                    "description": d["description"],
                    "organizationType": d["organizationType"],
                    "officeAddress": d["officeAddress"],
                    "placeOfPerformance": d["placeOfPerformance"],
                    "additionalInfoLink": d["additionalInfoLink"],
                    "uiLink": d["uiLink"],
                    "links": d["links"],
                    "resourceLinks": d["resourceLinks"],

                })
        print(len(list(i.properties)))
        client.close()
        

        

if __name__ == "__main__":
    folder_to_monitor = r"downloaded_data"

    # Set up the observer
    event_handler = FileMonitorHandler()
    observer = Observer()
    observer.schedule(event_handler, folder_to_monitor, recursive=False)

    print(f"Monitoring folder: {folder_to_monitor}")
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print("\nMonitoring stopped.")

    observer.join()


Monitoring folder: downloaded_data

Monitoring stopped.
